In [1]:
from iuvs import io

In [8]:
muvdarks = list(io.get_l1a_darks('muv'))

Searching for *muvdark*.fits.gz


In [9]:
from IPython.parallel import Client
c = Client()
lb = c.load_balanced_view()

In [10]:
def search_ff_dark(fname):
    from iuvs import io
    data = io.IUVS1AReader(str(fname))
    if data.img_header['NAXIS1'] == 1024:
        if data.img_header['NAXIS2'] == 1024:
            return fname

In [11]:
res = lb.map_async(search_ff_dark, muvdarks)

In [12]:
import time
while not res.ready():
    time.sleep(20)
    print(100*res.progress/len(muvdarks),'%')

13.248166548379466 %
22.64017033356991 %
32.00851667849539 %
41.51880766501065 %
49.491365034303286 %
57.771469127040454 %
66.31180506269222 %
74.94677075940383 %
84.10220014194464 %
93.75443577004968 %
100.0 %


In [13]:
import pandas as pd
s = pd.Series(res.result).dropna()

In [95]:
f = io.IUVS_Filename(s.iloc[0])

In [97]:
f.

'APP2'

In [64]:
f = IUVS_Filename(str(s.tolist()[0]))

In [65]:
f.tokens

['mvn',
 'iuv',
 'l1a',
 'IPH2-cycle00103-mode040-muvdark',
 '20140527T154453',
 'v00',
 'r00.fits.gz']

In [15]:
import os
s = s.map(lambda x: os.path.basename(str(x)))

In [16]:
s.sort()

In [17]:
df = pd.DataFrame(s)

In [18]:
df.columns = ['basename']

In [20]:
from iuvs.io import IUVS_Filename
df['op_phase'] = df.basename.map(lambda x: IUVS_Filename(x).cycle[:5])

In [21]:
df.op_phase.value_counts()

cycle    513
orbit      2
dtype: int64

In [22]:
def get_int_time(fname):
    path = io.level1apath / fname
    data = io.IUVS1AReader(str(path))
    return int(data.img_header['INT_TIME'])

In [23]:
df['INT_TIME'] = df.basename.map(get_int_time)

In [24]:
df.INT_TIME.value_counts()

60    257
6     256
14      2
dtype: int64

In [25]:
df.to_hdf('full_frame_muvs.h5','df')

In [26]:
%matplotlib qt

In [27]:
from matplotlib import rcParams
rcParams['image.interpolation'] = 'None'

In [120]:
equalled = []
fig, axes = plt.subplots(2,2)
axes = axes.flatten()
for ax,fname in zip(axes, [t6,t10,t14,t60]):
    print(fname)
    data = io.IUVS1AReader(fname)
    ax.imshow(data.img)
    ax.set_title("{}, {}, {}".format(data.fname.channel,
                                       data.fname.phase,
                                       data.img_header['INT_TIME']))

/maven_iuvs/stage/products/level1a/mvn_iuv_l1a_IPH2-cycle00022-mode050-muvdark_20140426T023038_v00_r00.fits.gz
/maven_iuvs/stage/products/level1a/mvn_iuv_l1a_early-orbit00003-mode003-muvdark_20140925T202812_v00_r00.fits.gz


TypeError: Invalid dimensions for image data

In [19]:
l1a_metadata = pd.read_table("/maven_iuvs/stage/products/level1a/IUVS_L1A_index_summary.txt",
              y)

In [57]:
ff = l1a_metadata[(l1a_metadata.NX == 1024) & (l1a_metadata.NY == 1024)]

In [56]:
l1a_metadata['is_dark'] = l1a_metadata.filename.map(lambda x: 'dark' in x)

In [89]:
ff[ff.is_dark][['INT_TIME','MCP_HV']]

,INT_TIME,MCP_HV
filenum,,
6610,60000,0
6625,14400,0
6626,14400,0
6627,14400,0
6628,14400,0


In [62]:
ff.index = ff.index.astype('int')

In [76]:
from matplotlib import rcParams
rcParams['image.interpolation'] = None

In [80]:
roi_indexes = [6610, 6627]

In [83]:
plt.colorbar?

In [99]:
fig, axes = plt.subplots(ncols=2, nrows=2)
for ax, ind in zip(axes.flatten()[:2], roi_indexes):
    fname_tokens = io.IUVS_Filename(ff.loc[ind].filename)
    fname = io.level1apath / ff.loc[ind].filename
    data = io.IUVS1AReader(str(fname))
    img = data.img #/ data.img_header['INT_TIME']
    im = ax.imshow(img)
    ax.set_title('{}, {}'.format(fname_tokens.channel, data.img_header['INT_TIME']))
    plt.colorbar(im, ax=ax)
for ax, ind in zip(axes.flatten()[2:], roi_indexes):
    fname_tokens = io.IUVS_Filename(ff.loc[ind].filename)
    fname = io.level1apath / ff.loc[ind].filename
    data = io.IUVS1AReader(str(fname))
    img = data.img / data.img_header['INT_TIME']
    im = ax.imshow(img)
    ax.set_title('{}, {}'.format(fname_tokens.channel, data.img_header['INT_TIME']))
    plt.colorbar(im, ax=ax)
fig.suptitle('Original darks, then scaled by integration time. (to DN/s)')
plt.savefig('Original_darks.png', dpi=150)

In [91]:
from skimage.exposure import rescale_intensity

In [100]:
fig, axes = plt.subplots(ncols=2, nrows=2)
for ax, ind in zip(axes.flatten()[:2], roi_indexes):
    fname_tokens = io.IUVS_Filename(ff.loc[ind].filename)
    fname = io.level1apath / ff.loc[ind].filename
    data = io.IUVS1AReader(str(fname))
    img = data.img / data.img_header['INT_TIME']
    flimg = rescale_intensity(1.0 * img)
    im = ax.imshow(flimg)
    ax.set_title('{}, {}'.format(fname_tokens.channel, data.img_header['INT_TIME']))
    plt.colorbar(im, ax=ax)
for ax, ind in zip(axes.flatten()[2:], roi_indexes):
    fname_tokens = io.IUVS_Filename(ff.loc[ind].filename)
    fname = io.level1apath / ff.loc[ind].filename
    data = io.IUVS1AReader(str(fname))
    img = data.img / data.img_header['INT_TIME']
    flimg = rescale_intensity(1.0 * img)
    im = ax.imshow(equalize_hist(flimg))
    ax.set_title('{}, {}'.format(fname_tokens.channel, data.img_header['INT_TIME']))
    plt.colorbar(im, ax=ax)
fig.suptitle('Stretched between 0 and 1. And then equalized_histogram.')
plt.savefig('Stretched_both_ways.png', dpi=150)

In [101]:
pwd

'/home/klay6683/src/iuvs/notebooks'

In [64]:
fnames = df[df.INT_TIME == 60].head(10).basename

In [2]:
%matplotlib qt

In [55]:
import matplotlib
matplotlib.style.use('ggplot')

In [80]:
import numpy as np
from skimage.exposure import (rescale_intensity, 
                              equalize_adapthist, 
                              equalize_hist)
import matplotlib.pyplot as plt
import skimage

def plot_stretched(fname, savename_prefix):
    fig, axes = plt.subplots(2,2)
    axes = axes.flatten()
    path = io.level1apath / fname
    data = io.IUVS1AReader(str(path))
    img = data.img
    im = axes[0].imshow(img)
    axes[0].set_title('Raw image, {} s'.format(data.img_header['INT_TIME']))
    plt.colorbar(im, ax=axes[0])
    p2, p98 = np.percentile(img, (2,98))
    img_rescale = rescale_intensity(img, in_range=(p2, p98))
    img_eq = equalize_hist(img)
#     img_ad_eq = equalize_adapthist(img, clip_limit=0.03)
    im = axes[1].imshow(img_rescale)
    axes[1].set_title('Normal rescale')
    plt.colorbar(im, ax=axes[1])
    im = axes[2].imshow(img_eq)
    axes[2].set_title('Hist equalized')
    plt.colorbar(im, ax=axes[2])
#     axes[3].imshow(img_ad_eq)
    fig.suptitle(os.path.basename(fname))
    plt.savefig(savename_prefix+'.png', dpi=150)
#     plt.close(fig)

In [82]:
for i,fname in enumerate(df[df.INT_TIME == 60].basename[:1]):
    plot_stretched(fname, 'dark_60s_{}'.format(str(i).zfill(2)))

In [79]:
for i,fname in enumerate(df[df.INT_TIME == 6].basename[:20]):
    plot_stretched(fname, 'dark_6s_{}'.format(str(i).zfill(2)))

In [84]:
len(muvdarks)

4227

In [88]:
ff = pd.read_hdf('full_frame_muvs.h5', 'df')

In [90]:
ff.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 515 entries, 467 to 1274
Data columns (total 3 columns):
basename    515 non-null object
op_phase    515 non-null object
INT_TIME    515 non-null int64
dtypes: int64(1), object(2)
memory usage: 16.1+ KB


In [104]:
def add_timestamp(fname):
    path = io.level1apath / fname
    iuvsfname = io.IUVS_Filename(str(path))
    return iuvsfname.time

In [94]:
ff.columns

Index(['basename', 'op_phase', 'INT_TIME'], dtype='object')

In [105]:
ff['time'] = ff.basename.map(add_timestamp)

In [107]:
ff.set_index('time', inplace=True)

In [109]:
ff.sort_index(inplace=True)

In [112]:
ff['fake'] = 1

In [123]:
%matplotlib qt

In [129]:
ff['has_cycle00116'] = ff.basename.str.contains('cycle00116')

In [132]:
ff[ff.has_cycle00116].basename.iloc[0]

'mvn_iuv_l1a_IPH2-cycle00116-mode040-muvdark_20140531T234520_v00_r00.fits.gz'

In [134]:
cd /maven_iuvs/stage/products/level1a/

/maven_iuvs/stage/products/level1a


In [137]:
fnames = !ls *IPH2-cycle00116*muv*.fits.gz*

In [143]:
ff[ff.basename.str.contains('IPH2')].basename.iloc[0]

'mvn_iuv_l1a_IPH2-cycle00001-mode040-muvdark_20140419T004311_v00_r00.fits.gz'

In [144]:
!ls *IPH2-cycle00001-mode040-muv*

mvn_iuv_l1a_IPH2-cycle00001-mode040-muv_20140419T004426_v00_r00.fits.gz
mvn_iuv_l1a_IPH2-cycle00001-mode040-muvdark_20140419T004311_v00_r00.fits.gz


In [147]:
ff['mode'] = ff.basename.map(lambda x: io.IUVS_Filename(x).mode)

In [149]:
ff.groupby('mode').size()

mode
mode040     257
mode050     256
mode2002      2
dtype: int64